
## **` Target Prediction for DMARDs as anti-Alzheimer’s Drugs`**






---
## **` Part I`**
---

In [1]:
# Install required libraries

%pip install --upgrade pip 
%pip install joblib
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install rdkit
%pip install scikit-learn==0.22  # Note that in this source code, the installed version of scikit-learn must be 0.22.
%pip install chembl_webresource_client
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import matplotlib as mpl
import joblib as joblib
import pandas as pd
import numpy as np
import requests
import tarfile
import csv

from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw
from rdkit import DataStructs
from rdkit.Chem import AllChem as Chem, Draw
from chembl_webresource_client.new_client import new_client

In [3]:
import pkg_resources

def report_installed_library_versions():
    libraries = [
        'joblib',
        'matplotlib',
        'pandas',
        'numpy',
        'rdkit',
        'scikit-learn',
        'chembl_webresource_client',
        'openpyxl'
    ]

    installed_packages = pkg_resources.working_set
    for package in installed_packages:
        if package.key in libraries:
            print(f"{package.key}=={package.version}")

report_installed_library_versions()

scikit-learn==0.22
rdkit==2023.3.3
pandas==2.0.3
openpyxl==3.1.2
numpy==1.24.4
matplotlib==3.7.3
joblib==1.3.2


In [4]:
# Configure matplotlib to display plots inline
%matplotlib inline
mpl.rcParams['figure.figsize'] = 10,10

In [5]:
# Download and extract the NB models
# URL of the NB models tar.gz file

should_run = False      #To run the code, type True instead of False
if should_run:

    url = 'https://ftp.ebi.ac.uk/pub/databases/chembl/target_predictions/NB/chembl_25_models.tar.gz'

    # Download the file
    response = requests.get(url)
    if response.status_code == 200:
        with open('chembl_25_models.tar.gz', 'wb') as file:
            file.write(response.content)
        print('File downloaded successfully.')
    else:
        print(f'Failed to download the file. Status code: {response.status_code}')

    # Extract the tar.gz file
    with tarfile.open('chembl_25_models.tar.gz', 'r:gz') as tar:
        tar.extractall()
print('File chembl_25_models downloaded successfully.')
print('File chembl_25_models extracted successfully.')


File chembl_25_models downloaded successfully.
File chembl_25_models extracted successfully.


In [6]:
# Load saved model using joblib
morgan_nb = joblib.load('E:/Thesis/materials and methods/chembl_25/models/10uM/mNB_10uM_all.pkl')

c:\Users\IRANIAN\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:313: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.19.1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\IRANIAN\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\Users\IRANIAN\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from jobl

---
## **` Part II`**
---

In [7]:
# Read the CSV file and create a dictionary of molecules (SMILES)
with open('E:/Thesis/materials and methods/DMARDs-Database-Smiles.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for rows in reader:
        moldict_1 = {row[0]:row[1] for row in reader}
    print(moldict_1)

    print('\n')

# Get the list of SMILEs strings from the dictionary
    smiles_list = list(moldict_1.values())
    print(smiles_list)
    
    

# Get the list of Chembl IDs from the dictionary
    mol_id = list(moldict_1.keys())
    print(mol_id)
    print(len(mol_id))
   

{'CHEMBL3655081': 'CCCS(=O)(=O)N[C@H]1C[C@@H](N(C)c2ncnc3[nH]ccc23)C1', 'CHEMBL3545241': 'Cc1cc(Nc2nc(C(O)c3ccc(F)cc3)nc3ccccc23)n[nH]1', 'CHEMBL3707348': 'CC#CC(=O)N1CCC[C@H]1c1nc(-c2ccc(C(=O)Nc3ccccn3)cc2)c2c(N)nccn12', 'CHEMBL1742420': 'CCOC(=O)NC(=N)c1ccc(OCc2cccc(COc3ccc(C(C)(C)c4ccc(O)cc4)cc3)c2)cc1', 'CHEMBL376180': 'Nc1nc(N)c2nc(CNc3ccc(C(=O)N[C@@H](CCC(=O)O)C(=O)O)cc3)cnc2n1', 'CHEMBL4297643': 'Cc1cccc(/C=N/Nc2cc(N3CCOCC3)nc(OCCc3ccccn3)n2)c1', 'CHEMBL206815': 'CC1(C)SCCN(S(=O)(=O)c2ccc(OCC#CCO)cc2)[C@H]1C(=O)NO', 'CHEMBL514800': 'CCOc1cc([C@@H](CS(C)(=O)=O)N2C(=O)c3cccc(NC(C)=O)c3C2=O)ccc1OC', 'CHEMBL1088750': 'CC(C)Cn1ncc2cc(Oc3ccc(F)cc3F)c(C(=O)NCCN(C)C)cc21', 'CHEMBL1542': 'Cn1cnc([N+](=O)[O-])c1Sc1ncnc2[nH]cnc12', 'CHEMBL1951914': 'CCN(C(=O)Cc1ccc(S(C)(=O)=O)cc1)C1CCN(CC[C@@H](c2cc(F)cc(F)c2)C2CCN(S(C)(=O)=O)CC2)CC1', 'CHEMBL3545108': 'O=C(NCC12CC3CC(CC(C3)C1)C2)c1cc(CCCNCCCO)ccc1Cl', 'CHEMBL2105759': 'CCS(=O)(=O)N1CC(CC#N)(n2cc(-c3ncnc4[nH]ccc34)cn2)C1', 'CHEMBL3187723':

In [8]:
# Read the CSV file and create a dictionary of molecules (NAME)
with open('E:/Thesis/materials and methods/DMARDs-Database-Names.csv', mode='r') as infile:
    reader = csv.reader(infile)

    for rows in reader:
        moldict_2 = {row[0]:row[1] for row in reader}
    print(moldict_2)

    print('\n')

# Get the list of molecule names from the dictionary
    mol_names_list = list(moldict_2.values())
    print(mol_names_list)
    print(len(mol_names_list))

{'CHEMBL3655081': 'ABROCITINIB', 'CHEMBL3545241': 'AC-430', 'CHEMBL3707348': 'ACALABRUTINIB', 'CHEMBL1742420': 'AMELUBANT', 'CHEMBL376180': 'AMINOPTERIN', 'CHEMBL4297643': 'APILIMOD', 'CHEMBL206815': 'APRATASTAT', 'CHEMBL514800': 'APREMILAST', 'CHEMBL1088750': 'ARRY-797', 'CHEMBL1542': 'AZATHIOPRINE', 'CHEMBL1951914': 'AZD5672', 'CHEMBL3545108': 'AZD9056', 'CHEMBL2105759': 'BARICITINIB', 'CHEMBL3187723': 'BINIMETINIB', 'CHEMBL1230065': 'BMS-582949', 'CHEMBL4297674': 'BRANEBRUTINIB', 'CHEMBL4297477': 'BREPOCITINIB', 'CHEMBL80830': 'BUCILLAMINE', 'CHEMBL4444976': 'CCX354', 'CHEMBL1823817': 'CE-224535', 'CHEMBL4596392': 'CRAVACITINIB', 'CHEMBL88': 'CYCLOPHOSPHAMIDE', 'CHEMBL3039513': 'DECERNOTINIB', 'CHEMBL4297507': 'DELGOCITINIB', 'CHEMBL2103838': 'DILMAPIMOD', 'CHEMBL2107333': 'DIMETHYL FUMARATE', 'CHEMBL103667': 'DORAMAPIMOD', 'CHEMBL3039502': 'DUVELISIB', 'CHEMBL3674570': 'EDICOTINIB', 'CHEMBL4594348': 'ELSUBRUTINIB', 'CHEMBL3358920': 'ETRASIMOD', 'CHEMBL4072833': 'EVOBRUTINIB', 'CHEM

In [9]:
# Create the output DataFrame
output_df = pd.DataFrame({
    'Chembl ID': mol_id,
    'Molecule Name': mol_names_list,
    'SMILES String': smiles_list
})


# Save the output table as an XLSX file
output_df.to_excel('output_table.xlsx', index=False)
print("Output table saved as 'output_table.xlsx'.")
output_df

Output table saved as 'output_table.xlsx'.


,Chembl ID,Molecule Name,SMILES String
0,CHEMBL3655081,ABROCITINIB,CCCS(=O)(=O)N[C@H]1C[C@@H](N(C)c2ncnc3[nH]ccc2...
1,CHEMBL3545241,AC-430,Cc1cc(Nc2nc(C(O)c3ccc(F)cc3)nc3ccccc23)n[nH]1
2,CHEMBL3707348,ACALABRUTINIB,CC#CC(=O)N1CCC[C@H]1c1nc(-c2ccc(C(=O)Nc3ccccn3...
3,CHEMBL1742420,AMELUBANT,CCOC(=O)NC(=N)c1ccc(OCc2cccc(COc3ccc(C(C)(C)c4...
4,CHEMBL376180,AMINOPTERIN,Nc1nc(N)c2nc(CNc3ccc(C(=O)N[C@@H](CCC(=O)O)C(=...
...,...,...,...
85,CHEMBL197194,VIDOFLUDIMUS,COc1cccc(-c2ccc(NC(=O)C3=C(C(=O)O)CCC3)c(F)c2)c1
86,CHEMBL1090090,VX-702,NC(=O)c1ccc(N(C(N)=O)c2c(F)cccc2F)nc1-c1ccc(F)...
87,CHEMBL5314423,ZASOCITINIB,CNc1cc(Nc2cccn(-c3ccccn3)c2=O)nc2c(C(=O)N[C@@H...
88,CHEMBL4081711,ZIMLOVISERTIB,CC[C@@H]1[C@H](F)C(=O)N[C@@H]1COc1nccc2cc(C(N)...


In [10]:
# Use RDKit to create a list of molecules from the SMILEs strings
mol_list = []
for smiles in smiles_list:
   mol = Chem.MolFromSmiles(smiles)
   mol_list.append(mol)
   img = Draw.MolsToGridImage(mol_list)

c:\Users\IRANIAN\AppData\Local\Programs\Python\Python38\lib\site-packages\rdkit\Chem\Draw\IPythonConsole.py:261: UserWarning: Truncating the list of molecules to be displayed to 50. Change the maxMols value to display more.
  warnings.warn(


In [11]:
# Get Morgan fingerprints for each molecule and predict the target class probabilities
classes = list(morgan_nb.targets)  # List of target classes
results = []  # List of dictionaries to store the top 20 probabilities and predicted classes for each molecule in mol_list

# Iterate over each molecule in mol_list
fingerprints = []
for mol in mol_list:
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048, bitInfo={})  # Get the Morgan fingerprint for the molecule
    res = np.zeros(len(fp), np.int32)
    DataStructs.ConvertToNumpyArray(fp, res)
    fingerprints.append(res)
    probas = morgan_nb.predict_proba(res.reshape(1, -1))[0]  # Predict the target class probabilities using the Morgan fingerprint

    top_20_indices = np.argsort(probas)[::-1][:20]  # Get the indices of the top 20 probabilities in descending order
    top_20_probas = probas[top_20_indices]  # Get the top 20 probabilities
    top_20_classes = [classes[i] for i in top_20_indices]  # Get the corresponding top 20 classes

    results.append({"proba": top_20_probas, "classes": top_20_classes})  # Add the top 20 probabilities and predicted classes to the results list

# Create a list of dictionaries that map class names to probabilities for each molecule
result_list = []
for result in results:
    class_dict = {}
    for i in range(len(result['classes'])):
        class_dict[result['classes'][i]] = result['proba'][i]
    result_list.append(class_dict)
# Print the result as a list of dictionaries
for i in range(len(result_list)):
    Ligand_name = mol_names_list[i]  # Get the name of the ligand
    print("{}.{} = {}".format(i + 1, Ligand_name, result_list[i]))  # Print the molecule index, ligand name, and the corresponding dictionary of class names and probabilities

1.ABROCITINIB = {'CHEMBL3553': 1.0, 'CHEMBL2148': 0.9999999999993179, 'CHEMBL2971': 0.9999999999810143, 'CHEMBL2835': 0.9999999999409397, 'CHEMBL5260': 0.999995655752514, 'CHEMBL5683': 0.978941800594929, 'CHEMBL4147': 0.9702929694699339, 'CHEMBL5073': 0.9538651634299264, 'CHEMBL3032': 0.9390515924354467, 'CHEMBL5699': 0.6686335656287574, 'CHEMBL1169596': 0.6001511711880749, 'CHEMBL2468': 0.2401615380113171, 'CHEMBL4924': 0.18490857497829533, 'CHEMBL4816': 0.1783254709087444, 'CHEMBL4577': 0.13447666688468235, 'CHEMBL4225': 0.13018896431761331, 'CHEMBL5932': 0.08557755769934583, 'CHEMBL5627': 0.055211412456764114, 'CHEMBL2964': 0.037583679541877385, 'CHEMBL5698': 0.03707312260504815}
2.AC-430 = {'CHEMBL4722': 0.9992668877727684, 'CHEMBL2185': 0.9973517991495714, 'CHEMBL1908385': 0.9957695770831015, 'CHEMBL4040': 0.9891405847927481, 'CHEMBL4203': 0.9734970516772286, 'CHEMBL3935': 0.9297315123113029, 'CHEMBL6166': 0.7501627698747351, 'CHEMBL4482': 0.700726074382782, 'CHEMBL4225': 0.465142

In [12]:
# Save the fingerprints to a CSV file
with open('fingerprints.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['SMILES'] + [f'Bit {i}' for i in range(2048)])
    for i, smiles in enumerate(smiles_list):
        row = [smiles] + list(fingerprints[i])
        writer.writerow(row)
print ("Data saved to fingerprints.csv")

Data saved to fingerprints.csv


In [13]:
len(classes)

1649

In [14]:
# Read the desired CSV file containing Alzheimer's targets and create a dictionary of Chembl IDs
with open('E:/Thesis/materials and methods/alz-target.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for rows in reader:
     target = {row[0]:row[1] for row in reader}
     print(target)

     print('\n')

     target_id_list = list(target.values())
     print(target_id_list)
     print(len(target_id_list))

{'Nrf2': 'CHEMBL1075094', 'LRRK2': 'CHEMBL1075104', 'PI3K': 'CHEMBL1075165', 'Proline-rich AKT1 substrate 1': 'CHEMBL1255161', 'NOX': 'CHEMBL1287627', 'Microtubule-associated protein tau': 'CHEMBL1293224', 'Beta-nerve growth factor': 'CHEMBL1649058', 'cyp D(Peptidyl-prolyl cis-trans isomerase D, mitochondrial)': 'CHEMBL1697657', 'NLRP3': 'CHEMBL1741208', 'IL18': 'CHEMBL1741305', 'Apoptotic protease-activating factor 1': 'CHEMBL1795093', 'IL6': 'CHEMBL1795129', 'Transforming growth factor beta-1': 'CHEMBL1795178', 'TNF alpha': 'CHEMBL1825', 'CD33': 'CHEMBL1842', 'RyR1': 'CHEMBL1846', 'IL1beta': 'CHEMBL1909490', 'butyrylcholinesterase': 'CHEMBL1914', 'Tau-tubulin kinase 1': 'CHEMBL1926492', 'DMT1': 'CHEMBL1932895', 'Cyclophilin A': 'CHEMBL1949', 'MAO A': 'CHEMBL1951', 'insuline receptor': 'CHEMBL1981', 'APP-BP1': 'CHEMBL2016431', 'MAO B': 'CHEMBL2039', 'Carbonic anhydrase II': 'CHEMBL205', 'PTEN': 'CHEMBL2052032', 'LPL': 'CHEMBL2060', 'RyR3': 'CHEMBL2062', 'Cyclophilin B': 'CHEMBL2075', 

In [16]:
# Separating Alzheimer's targets from all predicted targets
count = 0
keys_list = []
results_list = []

# Iterate over the result_list
for i, result in enumerate(result_list):
    for key, prob in result.items():  # Unpack key and probability directly
        if key in target_id_list:
            Ligand_name = mol_names_list[i]
            keys_list.append(key)
            results_list.append(prob)
            count += 1

# Print the results as a table
print('\n')
print('---------------------------------------------------------')
print(f"{'Ligand Name':<20} {'Target ID':<20} {'Probability':<15}")
print('---------------------------------------------------------')

# Use zip to print formatted results
for ligand_name, key, result in zip(mol_names_list, keys_list, results_list):
    print(f"{ligand_name:<20} {key:<20} {result:<15}")

print('---------------------')
print("Total data:", count)  # Print the total number of predicted targets



---------------------------------------------------------
Ligand Name          Target ID            Probability    
---------------------------------------------------------
ABROCITINIB          CHEMBL4040           0.9891405847927481
AC-430               CHEMBL4036           0.13231355838458883
ACALABRUTINIB        CHEMBL3227           0.00012227139474338988
AMELUBANT            CHEMBL279            9.178775153849155e-05
AMINOPTERIN          CHEMBL4040           2.7479564070987522e-05
APILIMOD             CHEMBL4179           1.731820217048015e-05
APRATASTAT           CHEMBL1293224        0.001357061288409064
APREMILAST           CHEMBL1892           0.8266714192517104
ARRY-797             CHEMBL1293224        0.9037668810945204
AZATHIOPRINE         CHEMBL3706           1.0            
AZD5672              CHEMBL321            0.9999999999992042
AZD9056              CHEMBL5028           0.9999769357501258
BARICITINIB          CHEMBL1293224        0.5418768247333067
BINIMETINIB      

In [17]:
# Assuming you have the following variables from the previous code:
# mol_list, mol_names_list, morgan_nb, classes, results, result_list
should_run = False      #To run the code, type True instead of False
if should_run:
 # Prompt the user for input
 target_chembl_id = input("Enter the ChEMBL ID target you are interested in: ")

 # Initialize an empty list to store the matching molecules
 matching_molecules = []

 # Iterate over the result_list and check if the target ChEMBL ID matches exactly
 for i, result_dict in enumerate(result_list):
     for class_name, probability in result_dict.items():
         if target_chembl_id == class_name.split()[0]:
             matching_molecules.append({
                 "Ligand_name": mol_names_list[i],
                 "ChEMBL_ID": target_chembl_id,
                 "Probability": probability
             })

 # Print the matching molecules
 if matching_molecules:
     print("Molecules with the target ChEMBL ID:")
     for molecule in matching_molecules:
        print(f"Ligand Name: {molecule['Ligand_name']}")
        print(f"ChEMBL ID: {molecule['ChEMBL_ID']}")
        print(f"Probability: {molecule['Probability']}")
        print()
 else:
    print(f"No molecules found with the target ChEMBL ID: {target_chembl_id}")

In [18]:
# Save the DataFrame to an Excel file
result_data = []
for i in range(len(result_list)):
    for key in result_list[i].keys():
        Ligand_name = mol_names_list[i]
        if key in target_id_list:
            result_data.append({
                'Ligand Number': i + 1,
                'Ligand Name': Ligand_name,
                'Target ID': key,
                'Proba': result_list[i][key]
            })

df = pd.DataFrame(result_data)
df.to_excel('Target_prediction.xlsx', index=False)
print('Data saved to Target_prediction.xlsx')

Data saved to Target_prediction.xlsx


In [19]:
def get_uniprot_id(target_id):    # Function to retrieve UniProt ID for a given target ID
    target = new_client.target
    target_result = target.get(target_id)
    components = target_result.get('target_components', [])
    for component in components:
        if 'accession' in component:
            return component['accession']
    return 'N/A'

def get_organism(target_id):    # Function to retrieve organism information for a given target ID
    target = new_client.target
    target_result = target.get(target_id)
    return target_result.get('organism', 'N/A')

def get_target_type(target_id):   # Function to retrieve target type information for a given target ID
    target = new_client.target
    target_result = target.get(target_id)
    return target_result.get('target_type', 'N/A')

In [20]:
high_prob_items = []  # List to store items with probability higher than 0.9
count = 0  # Counter variable to keep track of the total number of high probability items

# Iterate over mol_id list
for i, chembl_id_value in enumerate(mol_id):
    # Iterate over keys in result_list[i]
    for key in result_list[i]:
        # Check if the key is present in target_id_list and if the probability is greater than 0.9
        if key in target_id_list and result_list[i][key] > 0.9:
            target_name = new_client.target.get(key)['pref_name']
            uniprot_id = get_uniprot_id(key)  # Call function to get UniProt ID
            organism = get_organism(key)  # Call function to get organism information
            target_type = get_target_type(key)  # Call function to get target type
            # Append the item with relevant information to high_prob_items list
            high_prob_items.append({
                "molecule_number": i + 1,
                "molecule_id": chembl_id_value,
                "molecule_name": mol_names_list[i],
                "chemblid_of_target": key,
                "target_name": target_name,
                "uniprot_id": uniprot_id,
                "target_type": target_type,
                "organism": organism,
                "proba": result_list[i][key]
            })
            count += 1  # Increment the counter

In [21]:
# Print the items with probability higher than 0.9
print("Items with probability higher than 0.9:\n")
for item in high_prob_items:
    print("Molecule Number: {}\nMolecule ID: {}\nMolecule Name: {}\nTarget ID: {}\nTarget Name: {}\nUniProt ID: {}\nTarget Type: {}\nOrganism: {}\nProba: {}\n".format(
        item["molecule_number"],
        item["molecule_id"],
        item["molecule_name"],
        item["chemblid_of_target"],
        item["target_name"],
        item["uniprot_id"],
        item["target_type"],
        item["organism"],
        item["proba"]
    ))

print('\n')
print('---------------------')
print("Total data:", count)

Items with probability higher than 0.9:

Molecule Number: 2
Molecule ID: CHEMBL3545241
Molecule Name: AC-430
Target ID: CHEMBL4040
Target Name: MAP kinase ERK2
UniProt ID: P28482
Target Type: SINGLE PROTEIN
Organism: Homo sapiens
Proba: 0.9891405847927481

Molecule Number: 6
Molecule ID: CHEMBL4297643
Molecule Name: APILIMOD
Target ID: CHEMBL1293224
Target Name: Microtubule-associated protein tau
UniProt ID: P10636
Target Type: SINGLE PROTEIN
Organism: Homo sapiens
Proba: 0.9037668810945204

Molecule Number: 7
Molecule ID: CHEMBL206815
Molecule Name: APRATASTAT
Target ID: CHEMBL3706
Target Name: ADAM17
UniProt ID: P78536
Target Type: SINGLE PROTEIN
Organism: Homo sapiens
Proba: 1.0

Molecule Number: 7
Molecule ID: CHEMBL206815
Molecule Name: APRATASTAT
Target ID: CHEMBL321
Target Name: Matrix metalloproteinase 9
UniProt ID: P14780
Target Type: SINGLE PROTEIN
Organism: Homo sapiens
Proba: 0.9999999999992042

Molecule Number: 7
Molecule ID: CHEMBL206815
Molecule Name: APRATASTAT
Target I

In [22]:
# Save the DataFrame to an Excel file
df = pd.DataFrame(high_prob_items)
df.to_excel('high_prob_items_0.9.xlsx', index=False)

print('Data saved to high_prob_items_0.9.xlsx')

Data saved to high_prob_items_0.9.xlsx
